In [1]:
import pandas as pd
import time

In [2]:
start=time.perf_counter()

In [ ]:
#add this info to a settings dict?
stats_current_gameweek = 9
picks_current_gameweek = 12
current_season = 20252026

In [3]:
from DatabaseCreator import DatabaseCreator
import CJDH_local_settings

#Run Database Creator
if __name__ == "__main__":
    db_creator = DatabaseCreator(db_settings=CJDH_local_settings.local_settings['FPL_Points_Predictor'])
    fpl_engine = db_creator.get_engine_for("fpl_data_analysis")

In [ ]:
from FPL_League_Data_Scraper import FPL_League_Data_Scraper

league_code = "1446916"
season=2526

#Run Data Scraper for DivX league
if __name__ == "__main__":
    divx = FPL_League_Data_Scraper(league_code, season)
    divx_picksdf = divx.scrape_picks()

Data fetched, league name:  Riverside International League
Max gameweek data available for is:  12
6506490 Samia Shelim GW 1 Pick 1
6506490 Samia Shelim GW 1 Pick 2
6506490 Samia Shelim GW 1 Pick 3
6506490 Samia Shelim GW 1 Pick 4
6506490 Samia Shelim GW 1 Pick 5
6506490 Samia Shelim GW 1 Pick 6
6506490 Samia Shelim GW 1 Pick 7
6506490 Samia Shelim GW 1 Pick 8
6506490 Samia Shelim GW 1 Pick 9
6506490 Samia Shelim GW 1 Pick 10
6506490 Samia Shelim GW 1 Pick 11
6506490 Samia Shelim GW 1 Pick 12
6506490 Samia Shelim GW 1 Pick 13
6506490 Samia Shelim GW 1 Pick 14
6506490 Samia Shelim GW 1 Pick 15
6506490 Samia Shelim GW 2 Pick 1
6506490 Samia Shelim GW 2 Pick 2
6506490 Samia Shelim GW 2 Pick 3
6506490 Samia Shelim GW 2 Pick 4
6506490 Samia Shelim GW 2 Pick 5
6506490 Samia Shelim GW 2 Pick 6
6506490 Samia Shelim GW 2 Pick 7
6506490 Samia Shelim GW 2 Pick 8
6506490 Samia Shelim GW 2 Pick 9
6506490 Samia Shelim GW 2 Pick 10
6506490 Samia Shelim GW 2 Pick 11
6506490 Samia Shelim GW 2 Pick 12
6

In [13]:
#Add a new staging table & data into the database
divx_picks = divx_picksdf[['member', 'memberid','season', 'gw', 'pick', 
                 'element', 'position', 'multiplier', 
                 'is_captain', 'is_vice_captain']]

table_name = "divx_picks"
if __name__ == "__main__":
    db_creator.create_staging_table_then_insert_data(table_name, data=divx_picks)
    divx_picks = db_creator.table_to_df(table_name=table_name)
    print(divx_picks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3285 entries, 0 to 3284
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   member           3285 non-null   object 
 1   memberid         3285 non-null   float64
 2   season           3285 non-null   float64
 3   gw               3285 non-null   float64
 4   pick             3285 non-null   float64
 5   element          3285 non-null   float64
 6   position         3285 non-null   float64
 7   multiplier       3285 non-null   float64
 8   is_captain       3285 non-null   bool   
 9   is_vice_captain  3285 non-null   bool   
dtypes: bool(2), float64(7), object(1)
memory usage: 211.9+ KB
None


In [8]:
view = db_creator.run_sql(f"""
    SELECT
        player_name_id,
        position,
        team_name,
        value/10 as value,
        ROUND(AVG(opp_team_elo),1) AS avg_opp_team_elo,
        ROUND(SUM(xpoints),2) AS total_xpoints,
        ROUND(AVG(mins_pred),1) AS avg_mins_pred,
        ROUND(AVG(xgp90_pred),3) AS avg_xgp90_pred,
        ROUND(AVG(xap90_pred),3) AS avg_xap90_pred,
        ROUND(AVG(xgcp90_pred),3) AS avg_xgcp90_pred
    FROM predictions
    WHERE event BETWEEN {picks_current_gameweek} AND {picks_current_gameweek+5}
        -- AND position = 'MID'
        -- AND value <= 48
    GROUP BY player_name_id, position, team_name, value
    ORDER BY total_xpoints DESC
""")
view.head(20)


,player_name_id,position,team_name,value,avg_opp_team_elo,total_xpoints,avg_mins_pred,avg_xgp90_pred,avg_xap90_pred,avg_xgcp90_pred
0,Jean-Philippe Mateta,FWD,Crystal Palace,7.6,1008.7,38.31,73.8,0.949,0.101,1.327
1,Bryan Mbeumo,MID,Man Utd,8.0,1044.4,35.80,81.3,0.443,0.219,1.475
2,Mohamed Salah,MID,Liverpool,14.3,1011.4,35.70,77.0,0.438,0.237,1.163
3,Erling Haaland,FWD,Man City,14.7,1024.1,35.12,72.4,0.761,0.091,1.273
4,Gabriel dos Santos MagalhÃ£es,DEF,Arsenal,6.4,1056.7,33.72,84.0,0.134,0.062,0.872
5,Bruno Borges Fernandes,MID,Man Utd,8.9,1044.4,33.42,78.6,0.350,0.248,1.454
6,JurriÃ«n Timber,DEF,Arsenal,6.0,1056.7,33.32,77.4,0.158,0.085,0.900
7,Daniel MuÃ±oz MejÃ­a,DEF,Crystal Palace,5.6,1008.7,32.21,84.0,0.059,0.235,1.079
8,Riccardo Calafiori,DEF,Arsenal,5.8,1056.7,31.42,66.8,0.121,0.049,0.861
9,Matheus Nunes,DEF,Man City,5.3,1024.1,30.45,75.7,0.059,0.069,0.884


In [14]:
divx_picks.head()

,member,memberid,season,gw,pick,element,position,multiplier,is_captain,is_vice_captain
0,Samia Shelim,6506490.0,2526.0,1.0,1.0,220.0,1.0,1.0,False,True
1,Samia Shelim,6506490.0,2526.0,1.0,2.0,38.0,2.0,1.0,False,False
2,Samia Shelim,6506490.0,2526.0,1.0,3.0,5.0,3.0,1.0,False,False
3,Samia Shelim,6506490.0,2526.0,1.0,4.0,568.0,4.0,1.0,False,False
4,Samia Shelim,6506490.0,2526.0,1.0,5.0,53.0,5.0,1.0,False,False


In [ ]:
view = db_creator.run_sql(f"""
    SELECT *     
    FROM divx_picks
    LEFT JOIN playergw ON divx_picks.element = playergw.element
                          AND divx_picks.season = playergw.season -- 2526 vs 20252026
                          AND divx_picks.gw = playergw.event
""")
view.head(20)

# db_creator.run_sql("""
#     UPDATE divx_picks
#     SET season = 
#         CASE
#             WHEN LENGTH(season::text) = 4 THEN
#                 CONCAT('20', LEFT(season::text, 2), '20', RIGHT(season::text, 2))
#             ELSE season
#         END
# """)



,member,memberid,season,gw,pick,element,position,multiplier,is_captain,is_vice_captain,...,expected_goals,expected_goals_conceded,starts,cbi,defensive_contribution,recoveries,tackles,saves,team_name,opp_team_name
0,Samia Shelim,6506490.0,2526.0,1.0,12.0,470.0,12.0,0.0,False,False,...,None,None,None,None,None,None,None,None,None,None
1,Samia Shelim,6506490.0,2526.0,1.0,13.0,447.0,13.0,0.0,False,False,...,None,None,None,None,None,None,None,None,None,None
2,Samia Shelim,6506490.0,2526.0,2.0,3.0,568.0,3.0,1.0,False,False,...,None,None,None,None,None,None,None,None,None,None
3,Samia Shelim,6506490.0,2526.0,2.0,8.0,382.0,8.0,1.0,False,False,...,None,None,None,None,None,None,None,None,None,None
4,Samia Shelim,6506490.0,2526.0,2.0,10.0,430.0,10.0,2.0,True,False,...,None,None,None,None,None,None,None,None,None,None
5,Samia Shelim,6506490.0,2526.0,2.0,13.0,235.0,13.0,0.0,False,False,...,None,None,None,None,None,None,None,None,None,None
6,Samia Shelim,6506490.0,2526.0,2.0,14.0,38.0,14.0,0.0,False,False,...,None,None,None,None,None,None,None,None,None,None
7,Samia Shelim,6506490.0,2526.0,4.0,6.0,427.0,6.0,1.0,False,False,...,None,None,None,None,None,None,None,None,None,None
8,Samia Shelim,6506490.0,2526.0,4.0,14.0,191.0,14.0,0.0,False,False,...,None,None,None,None,None,None,None,None,None,None
9,Samia Shelim,6506490.0,2526.0,4.0,15.0,100.0,15.0,0.0,False,False,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# max_gw = max(picksdf['gw'])
# me = 'Christopher Harris'
# myid = set(picksdf[picksdf['member']==me]['memberid'])
# mypicksdf = picksdf[(picksdf['member']==me)&(picksdf['gw']==max_gw)]
# mypicks = set(mypicksdf['element'])
# mypicks

# #league_standings
# l = pd.DataFrame(league_data['standings']['results'])
# mytotal = l[l['entry'].isin(myid)]['total'].values
# #get ids for members above me or less than 100 points below me
# ids_close_to_me = set(l[l['total']<=int(mytotal)+100]['entry'][~l['entry'].isin(myid)])

In [ ]:
# #Get ep_next * by ownership rate in mini-league
# latestpicks = picksdf[(picksdf['gw']==max(picksdf['gw']))&(picksdf['memberid'].isin(ids_close_to_me))]
# max_owns = len(set(latestpicks['memberid']))

# ownership = pd.pivot_table(latestpicks, index=['element'],values='multiplier',aggfunc=np.sum).reset_index()
# ownership['own_rate'] = ownership['multiplier']/max_owns

# ownership.sort_values(by='own_rate', ascending=False)

# playerdf = pd.merge(playerdf, ownership, how='outer',on='element')
# filtered = playerdf['element'].isin(mypicks)
# gainpicks = playerdf[filtered].merge(mypicksdf,how='left',on='element',suffixes=('','_myteam'))

# gainpicks['expected_gain'] = gainpicks['ep_next'].astype(float)*(1-gainpicks['own_rate']) #change 1 for "gainpicks['multiplier_myteam']" if team multiplier needed
# gainpicks[['web_name','element','element_type','multiplier','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False) #'multiplier_myteam'

In [ ]:
# print("Expected Total Gain on Division Average")
# gainpicks['expected_gain'].sum()

In [ ]:
# #players not in my picks
# losspicks = playerdf[~filtered].copy().dropna(subset='multiplier')
# losspicks['expected_loss'] = losspicks['ep_next'].astype(float)*(0-losspicks['own_rate'])
# print("Expected Total Loss on Division Average: ")
# print(losspicks['expected_loss'].sum())
# losspicks[['web_name','element','multiplier','own_rate','ep_next','expected_loss']].sort_values(by='expected_loss',ascending=True).head()

In [ ]:
# print("Net Gain")
# print(round(gainpicks['expected_gain'].sum() + losspicks['expected_loss'].sum(),2))

In [ ]:
# teamgains = gainpicks[['web_name','status','element',
#                        'element_type','now_cost','multiplier',
#                        'multiplier_myteam','own_rate','ep_next','expected_gain']].sort_values(by='expected_gain',ascending=False)
# teamgains['expected_loss_on_transfer'] = teamgains['ep_next'].astype(float)*(teamgains['own_rate'])

# allgains = playerdf.copy()
# allgains['expected_gain'] = allgains['ep_next'].astype(float)*(1-allgains['own_rate'])

# def top_suggested_transfer(row, df, current_team):
#     #get element, element_type & gain
#     myteam_type = row['element_type']
#     # myteam_gain = row['expected_gain']
#     myteam_loss = row['expected_loss_on_transfer']
#     myteam_cost = row['now_cost']
#     myteam_element = row['element']
#     #get playerdf element, element_type & gain
#     filtered = (df['element_type']==myteam_type) & (df['expected_gain']>myteam_loss) & (~df['element'].isin(current_team)) & (df['now_cost']<=myteam_cost)
#     best_gain = df[filtered].sort_values(by='expected_gain', ascending=False).head(1)
#     best_gain['own_rate'] = round(best_gain['own_rate'],3)
#     best_gain['expected_gain'] = round(best_gain['expected_gain'],2)
#     best_gain['diff'] = round(df['expected_gain']-row['expected_loss_on_transfer'],2)
#     return best_gain[['web_name','status','own_rate','ep_next','expected_gain','diff']].values[0]

# current_team = set(gainpicks['element'])
# teamgains['affordable_suggested_transfer'] = teamgains.apply(lambda x: top_suggested_transfer(x, allgains, current_team),axis=1)
# # Extracting the second element from the arrays in the "affordable_suggested_transfer" column
# teamgains['suggested_transfer_gain'] = teamgains['affordable_suggested_transfer'].apply(lambda x: x[-1])

# # Sorting the DataFrame based on the second element in the arrays
# teamgains['gain_diff'] = teamgains['suggested_transfer_gain'] - teamgains['expected_gain'] - teamgains['expected_loss_on_transfer']
# teamgains = teamgains.sort_values(by='gain_diff', ascending=False)

# teamgains.head(20)

In [ ]:
# print("Sorted by Expected Gain")
# allgains[['web_name','status','element','element_type','now_cost','ep_next','own_rate','expected_gain']].sort_values(by='expected_gain',ascending=False).dropna().head(20)

In [6]:
finish=time.perf_counter()
print(f'Finished in {round(((finish-start)/60),2)} minute(s)')
#10/11/25: Finished in 0.01 minute(s)

Finished in 0.02 minute(s)
